In [2]:
from openai import OpenAI
import numpy as np

In [11]:
client = OpenAI(api_key="") # I took out the API key because I could not push it to Github

In [6]:
with open("messi.txt", "r") as f:
    text = f.read()
    print(text)

Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi set numerous records for individual accolades won throughout his professional footballing career such as eight Ballon d'Or awards and eight times being named the world's best player by FIFA.[note 2] He is the most decorated player in the history of professional football having won 45 team trophies,[note 3] including twelve league titles, four UEFA Champions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds the records for most European Golden Shoes (6), most goals for a single club (672, with Barcelona), most goals (474), hat-tricks (36) and assists (192) in La Liga, most matches played (39), assists (18) and goal contributions (34) 

In [7]:
CHUNK_SIZE = 100
chunks = []
current_place = 0
while current_place < len(text):
    current_chunk = text[current_place : current_place + CHUNK_SIZE]
    chunks.append(current_chunk)
    current_place += CHUNK_SIZE
print(chunks)

['Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987), a', 'lso known as Leo Messi, is an Argentine professional footballer who plays as a forward for and capta', 'ins both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as on', 'e of the greatest players of all time, Messi set numerous records for individual accolades won throu', "ghout his professional footballing career such as eight Ballon d'Or awards and eight times being nam", "ed the world's best player by FIFA.[note 2] He is the most decorated player in the history of profes", 'sional football having won 45 team trophies,[note 3] including twelve league titles, four UEFA Champ', 'ions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds the records for most European G', 'olden Shoes (6), most goals for a single club (672, with Barcelona), most goals (474), hat-tricks (3', '6) and assists (192) in La Liga, most matches played (39), ass

In [12]:
embedded_chunks = []
for chunk in chunks:
    response = client.embeddings.create(
        input=chunk,
        model="text-embedding-3-small"
    )
    embedded_chunks.append( (chunk, response.data[0].embedding) )
print(embedded_chunks)

[('Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987), a', [0.024370167404413223, -0.043210968375205994, -0.014181798323988914, -0.001041875802911818, -0.0007001303019933403, -0.0003910234081558883, 0.03546985611319542, 0.027913056313991547, -0.017038637772202492, -0.004710200242698193, -0.06291189044713974, 2.835879786289297e-05, -0.004682041704654694, -0.0248411875218153, 0.026274725794792175, 0.036780521273612976, -0.06975192576646805, 0.0008242850308306515, -0.016915762796998024, -0.04276042804121971, 0.05762827768921852, -0.0038654361851513386, 0.0037732799537479877, 0.0026878861244767904, 0.019199185073375702, 0.007444164250046015, -0.03905370458960533, 0.0031845050398260355, -0.01142735593020916, 0.020192423835396767, 0.07089875638484955, -0.019526852294802666, -0.0017701649339869618, -0.02069416269659996, 0.02377627231180668, 0.010577471926808357, -0.0011301920749247074, -0.014212517067790031, 0.0006476525450125337, -0.030984926968

In [19]:
embedded_chunks_dict = {embedded_chunk[0]: embedded_chunk[1] for embedded_chunk in embedded_chunks}

In [13]:
def cosine_simularity(A, B):
    return np.dot(A,B) / ( np.linalg.norm(A) * np.linalg.norm(B) )

In [14]:
question = "Why is Lionel Messi famous"

In [15]:
question_embedding = client.embeddings.create(
        input=question,
        model="text-embedding-3-small"
    ).data[0].embedding
print(question_embedding)

[0.03389046713709831, -0.04988497868180275, -0.052793070673942566, 0.03181006386876106, 0.007549856789410114, -0.023801621049642563, -0.0193388182669878, 0.06621503829956055, 0.004739632364362478, -0.033398330211639404, -0.0690336525440216, 0.023756882175803185, -0.03004283644258976, -0.026262316852808, -0.010251027531921864, 0.0056008752435445786, -0.07274705916643143, 0.011576446704566479, -0.00524295587092638, -0.02130737341940403, 0.0036183386109769344, 0.028051910921931267, 0.001700115855783224, 0.004896221682429314, 0.033286478370428085, 0.015569481998682022, -0.032660119235515594, -0.010278989560902119, -0.0010199296521022916, 0.04031063988804817, 0.04840856045484543, -0.024226650595664978, -0.008808165788650513, -0.016195841133594513, -0.030467865988612175, -0.009037458337843418, 0.006845203228294849, -0.009786850772798061, 0.02017769031226635, -0.0198980662971735, -0.01053624413907528, -0.06321746855974197, 0.02147514745593071, 0.02729133330285549, -0.00839991495013237, -0.041

In [20]:
csdict = {}
for chunk in embedded_chunks_dict:
    similarity = cosine_simularity(question_embedding, embedded_chunks_dict[chunk])
    csdict[chunk] = similarity
top_5 = ". ".join(sorted(csdict.keys(), key = lambda x: csdict[x])[-5:])


In [21]:
print(f"Relevant information: {top_5}")

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user", "content": f"User question: {question}. Potentially useful information: {top_5}"
        }
    ]
)

Relevant information: lso known as Leo Messi, is an Argentine professional footballer who plays as a forward for and capta. eam-sport athlete to surpass $1 billion in career earnings.

Early life
Messi was born on 24 June 19. e of the greatest players of all time, Messi set numerous records for individual accolades won throu. ational, Messi is the national team's all-time leading goalscorer and most-capped player. His style . Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987), a


In [22]:
print(completion.choices[0].message.content)

Lionel Messi is famous for being one of the greatest footballers of all time. He has achieved remarkable success both individually and with his teams. Here are some key reasons why Messi is so famous:

1. **Individual Achievements**: Messi has won numerous awards, including multiple FIFA Ballon d'Or awards, which are given to the world's best player annually. He has set and broken various records for goals scored in a calendar year, individual matches, and seasons.

2. **Club Success**: Messi spent the majority of his career at FC Barcelona, where he became the all-time top scorer for the club. During his time at Barcelona, he helped the team win numerous domestic and international titles, including the UEFA Champions League and La Liga.

3. **National Team Contributions**: As part of Argentina's national team, Messi is the all-time leading goalscorer and has won major tournaments, such as the Copa América.

4. **Playing Style**: Messi is renowned for his dribbling ability, vision, and